# Proyecto 200Eng - Black list



### Importa dependencias

In [4]:
# pip install pymysql

In [5]:
#importa dependencias
from pymysql import connect
import pandas as pd
from datetime import datetime, date, time, timedelta
from textwrap import dedent
import matplotlib.pyplot as plt
from millify import millify, prettify
import os

In [6]:
from millify import millify

print(millify(1_234_123_123, precision=2, prefixes=[' miles', ' millones', ' miles de millones']))
# '1k'

print(millify('1234')) # same for strings)
# '1k'

print(millify(12345678))
# '12M'

print(millify(12345678, precision=2))
# '12.35M'

print(millify(10000, precision=2)) # hide nulls in decimals by default
# '10k'

print(millify(10000, precision=2, drop_nulls=False))
# '10.00k'

prefixes = ['kB', 'MB', 'GB']
print(millify(10_000_000, prefixes=prefixes))
# '10kB'

1.23 miles de millones
1k
12M
12.35M
10k
10.00k
10MB


In [7]:
datetime.now()

datetime.datetime(2024, 9, 17, 9, 10, 46, 812195)

In [8]:
datetime.now() - timedelta(days=30)

datetime.datetime(2024, 8, 18, 9, 10, 46, 822238)

### Conexión con todos los marcadores

In [9]:
# lista de marcadores
url_marcadores = ['83.138.55.125',
              '193.219.97.92',
              '193.219.97.90',
              '83.138.55.115',
              '94.177.9.69',
              '83.138.55.163',
              ]

ind_marcadores = [1, 2, 3, 4, 6, 7]

In [10]:
# creación de un diccionario de diccionarios para cada marcadro

bd = {}
cursores = {}

for i, ind in enumerate(ind_marcadores):
    # crea la conexión
    bd[f'marcador_{ind}'] = connect(host = url_marcadores[i],
                                                user='carlosai',
                                                passwd='OKJmBUIFnh',
                                                database='mbilling',
                                                )

    # crea el cursor
    cursores[f'cursor_marcador_{ind}'] = bd[f'marcador_{ind}'].cursor()


In [11]:
bd

{'marcador_1': <pymysql.connections.Connection at 0x7b2c966e0a40>,
 'marcador_2': <pymysql.connections.Connection at 0x7b2c9656e3c0>,
 'marcador_3': <pymysql.connections.Connection at 0x7b2c9656e540>,
 'marcador_4': <pymysql.connections.Connection at 0x7b2c9656e4b0>,
 'marcador_6': <pymysql.connections.Connection at 0x7b2c9656e600>,
 'marcador_7': <pymysql.connections.Connection at 0x7b2c9656e780>}

In [12]:
cursores

{'cursor_marcador_1': <pymysql.cursors.Cursor at 0x7b2c9667a270>,
 'cursor_marcador_2': <pymysql.cursors.Cursor at 0x7b2c9656e390>,
 'cursor_marcador_3': <pymysql.cursors.Cursor at 0x7b2c9656ddc0>,
 'cursor_marcador_4': <pymysql.cursors.Cursor at 0x7b2c9656d940>,
 'cursor_marcador_6': <pymysql.cursors.Cursor at 0x7b2c9656df40>,
 'cursor_marcador_7': <pymysql.cursors.Cursor at 0x7b2c9656e6c0>}

### Extracción de datos

In [13]:
# rango de tres meses
now = datetime.now()
now_menos_90 = (now - timedelta(days=90)).strftime('%Y-%m-%d')
restart = '2024-08-01'

if restart > now_menos_90:
    date_ref = restart
else:
    date_ref = now_menos_90

print(f'Fecha de referencia: {date_ref}')


Fecha de referencia: 2024-08-01


In [14]:
# define la consulta (query)
query = dedent(f"""
        SELECT starttime, calledstation, real_sessiontime
        FROM pkg_cdr
        WHERE starttime > '{date_ref}';
        """)

print(f'Query a correr:\n{query}')


Query a correr:

SELECT starttime, calledstation, real_sessiontime
FROM pkg_cdr
WHERE starttime > '2024-08-01';



In [15]:
# crea una función para devolver cada df
def get_df(bd, bd_name):
    df_temp = pd.read_sql(
        query,
        bd,
        )
    df_temp['bd'] = bd_name

    return df_temp

In [16]:
#lista de df
df_list = [get_df(bd[f'marcador_{ind}'], f'marcador_{ind}') for ind in ind_marcadores]

/tmp/ipykernel_437213/147928526.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_temp = pd.read_sql(
/tmp/ipykernel_437213/147928526.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_temp = pd.read_sql(
/tmp/ipykernel_437213/147928526.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_temp = pd.read_sql(
/tmp/ipykernel_437213/147928526.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Ple

In [17]:
df_list[0]

,starttime,calledstation,real_sessiontime,bd
0,2024-08-06 13:16:09,18884405890,10,marcador_1
1,2024-08-06 13:17:10,18556031153,107,marcador_1
2,2024-08-06 13:20:07,18776831341,103,marcador_1
3,2024-08-06 13:21:06,18776290742,38,marcador_1
4,2024-08-06 13:23:07,18663131697,3,marcador_1
...,...,...,...,...
37980,2024-08-08 21:14:50,18448566237,91,marcador_1
37981,2024-08-08 21:15:09,18555588433,219,marcador_1
37982,2024-08-08 21:15:50,18004888085,319,marcador_1
37983,2024-08-08 21:16:07,18775415678,33,marcador_1


In [18]:
df = pd.concat(df_list, axis=0)
df

/tmp/ipykernel_437213/1100062364.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(df_list, axis=0)


,starttime,calledstation,real_sessiontime,bd
0,2024-08-06 13:16:09,18884405890,10,marcador_1
1,2024-08-06 13:17:10,18556031153,107,marcador_1
2,2024-08-06 13:20:07,18776831341,103,marcador_1
3,2024-08-06 13:21:06,18776290742,38,marcador_1
4,2024-08-06 13:23:07,18663131697,3,marcador_1
...,...,...,...,...
89464,2024-08-14 19:00:18,18446176567,2016,marcador_3
89465,2024-08-14 19:00:31,18552985095,2016,marcador_3
89466,2024-08-14 19:00:43,18559382318,2016,marcador_3
89467,2024-08-14 19:00:55,18772877762,2016,marcador_3


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 322563 entries, 0 to 89468
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   starttime         322563 non-null  datetime64[ns]
 1   calledstation     322563 non-null  object        
 2   real_sessiontime  322563 non-null  object        
 3   bd                322563 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 12.3+ MB


In [20]:
df['weekday'] = df.starttime.dt.weekday
df['hour'] = df.starttime.dt.hour
df

,starttime,calledstation,real_sessiontime,bd,weekday,hour
0,2024-08-06 13:16:09,18884405890,10,marcador_1,1,13
1,2024-08-06 13:17:10,18556031153,107,marcador_1,1,13
2,2024-08-06 13:20:07,18776831341,103,marcador_1,1,13
3,2024-08-06 13:21:06,18776290742,38,marcador_1,1,13
4,2024-08-06 13:23:07,18663131697,3,marcador_1,1,13
...,...,...,...,...,...,...
89464,2024-08-14 19:00:18,18446176567,2016,marcador_3,2,19
89465,2024-08-14 19:00:31,18552985095,2016,marcador_3,2,19
89466,2024-08-14 19:00:43,18559382318,2016,marcador_3,2,19
89467,2024-08-14 19:00:55,18772877762,2016,marcador_3,2,19


In [21]:
df = df.set_index('starttime')
df = df.sort_index()
df

,calledstation,real_sessiontime,bd,weekday,hour
starttime,,,,,
2024-08-06 13:16:09,18884405890,10,marcador_1,1,13
2024-08-06 13:17:10,18556031153,107,marcador_1,1,13
2024-08-06 13:20:07,18776831341,103,marcador_1,1,13
2024-08-06 13:21:06,18776290742,38,marcador_1,1,13
2024-08-06 13:23:07,18663131697,3,marcador_1,1,13
...,...,...,...,...,...
2024-08-14 19:01:47,18448903265,103,marcador_2,2,19
2024-08-14 19:01:47,18008076542,417,marcador_3,2,19
2024-08-14 19:01:47,18882544416,157,marcador_2,2,19


In [22]:
df.index.nunique()

187895

In [23]:
df.shape

(322563, 5)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 322563 entries, 2024-08-06 13:16:09 to 2024-08-14 19:01:56
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   calledstation     322563 non-null  object
 1   real_sessiontime  322563 non-null  object
 2   bd                322563 non-null  object
 3   weekday           322563 non-null  int32 
 4   hour              322563 non-null  int32 
dtypes: int32(2), object(3)
memory usage: 20.4+ MB


Observaciones
* La columna `callerid` es igual a la columna `calleddestination`

In [25]:
# import os
# cwd = os.getcwd()

In [26]:
# %pwd

In [27]:
# df.to_csv('llamadas_2024-08-01.csv')

In [28]:
df.nunique().sort_values(ascending=False)

calledstation       50315
real_sessiontime     3655
hour                   15
weekday                 7
bd                      3
dtype: int64

In [29]:
df_dest_calls = df.calledstation.value_counts().to_frame()
df_dest_calls = df_dest_calls.rename(columns={'count': 'num_llamadas'})
df_dest_calls

,num_llamadas
calledstation,
18884727846,9
18775120619,9
18556733195,9
18669798255,9
18006002413,9
...,...
18884811346,1
18772081290,1
18885302154,1


### Análisis de información

In [30]:
df_dest = df.groupby(['calledstation']).real_sessiontime.sum().sort_values(ascending=False).to_frame()
df_dest

,real_sessiontime
calledstation,
18005514636,21257
18003703848,21144
18558856518,21106
18007456973,21057
18776981885,20930
...,...
18885302154,2
18882430835,2
18772912512,2


In [31]:
df_dest = df_dest.join(df_dest_calls, on='calledstation')
df_dest

,real_sessiontime,num_llamadas
calledstation,,
18005514636,21257,7
18003703848,21144,7
18558856518,21106,7
18007456973,21057,7
18776981885,20930,7
...,...,...
18885302154,2,1
18882430835,2,1
18772912512,2,1


In [32]:
df_dest['dur_prom'] = df_dest.real_sessiontime / df_dest.num_llamadas
df_dest

,real_sessiontime,num_llamadas,dur_prom
calledstation,,,
18005514636,21257,7,3036.714286
18003703848,21144,7,3020.571429
18558856518,21106,7,3015.142857
18007456973,21057,7,3008.142857
18776981885,20930,7,2990.0
...,...,...,...
18885302154,2,1,2.0
18882430835,2,1,2.0
18772912512,2,1,2.0


In [33]:
df_dest.sort_values(by='num_llamadas', ascending=False)

,real_sessiontime,num_llamadas,dur_prom
calledstation,,,
18775120619,1010,9,112.222222
18669798255,4827,9,536.333333
18884727846,903,9,100.333333
18556733195,3259,9,362.111111
18885624401,7740,9,860.0
...,...,...,...
18444504718,60,1,60.0
18555560171,38,1,38.0
18884985288,317,1,317.0


In [34]:
df_dest.index

Index(['18005514636', '18003703848', '18558856518', '18007456973',
       '18776981885', '18552277612', '18557381992', '18558696548',
       '18779168687', '18444353069',
       ...
       '18668331776', '18556999015', '18662779476', '18883089334',
       '18663992605', '18885302154', '18882430835', '18772912512',
       '18444786683', '18884557342'],
      dtype='object', name='calledstation', length=50315)

In [35]:
def P5(x):
    return x.quantile(0.05)

def P25(x):
    return x.quantile(0.25)

def P50(x):
    return x.quantile(0.5)

def P75(x):
    return x.quantile(0.75)

def P95(x):
    return x.quantile(0.95)

In [36]:
%%time
df_dest_stats = df.groupby('calledstation').aggregate({'real_sessiontime': ['max', 'min', P25, P50, P75]})


CPU times: user 1min 11s, sys: 7.16 ms, total: 1min 11s
Wall time: 1min 11s


In [37]:
df_dest_stats

real_sessiontime                              
                           max  min     P25     P50      P75
calledstation                                               
18002001956                595  583  590.00   593.0   594.50
18002002000                 40   38   39.00    39.0    39.50
18002002599                214  108  115.00   124.5   164.75
18002002665               3601  721  810.50  1324.0  2719.50
18002003111                194   20   48.50    52.0    52.00
...                        ...  ...     ...     ...      ...
18889995252                218   55   55.25   134.5   216.00
18889996569                269  249  252.00   253.0   253.00
18889999101                324  318  319.00   319.0   319.00
18889999353                456   58   70.00    83.0   453.00
18889999520                291   27   32.50   211.0   290.50

[50315 rows x 5 columns]

In [38]:
df_dest_stats.columns = df_dest_stats.columns.map('_'.join).str.strip('_')
df_dest_stats

,real_sessiontime_max,real_sessiontime_min,real_sessiontime_P25,real_sessiontime_P50,real_sessiontime_P75
calledstation,,,,,
18002001956,595,583,590.00,593.0,594.50
18002002000,40,38,39.00,39.0,39.50
18002002599,214,108,115.00,124.5,164.75
18002002665,3601,721,810.50,1324.0,2719.50
18002003111,194,20,48.50,52.0,52.00
...,...,...,...,...,...
18889995252,218,55,55.25,134.5,216.00
18889996569,269,249,252.00,253.0,253.00
18889999101,324,318,319.00,319.0,319.00


In [39]:
df_dest = df_dest.join(df_dest_stats, on='calledstation')
df_dest

,real_sessiontime,num_llamadas,dur_prom,real_sessiontime_max,real_sessiontime_min,real_sessiontime_P25,real_sessiontime_P50,real_sessiontime_P75
calledstation,,,,,,,,
18005514636,21257,7,3036.714286,3600,1897,2486.0,3593.0,3597.5
18003703848,21144,7,3020.571429,3600,1767,2490.5,3598.0,3599.0
18558856518,21106,7,3015.142857,3601,1774,2467.0,3598.0,3599.5
18007456973,21057,7,3008.142857,3602,1661,2501.5,3593.0,3599.0
18776981885,20930,7,2990.0,3601,1662,2435.5,3598.0,3599.0
...,...,...,...,...,...,...,...,...
18885302154,2,1,2.0,2,2,2.0,2.0,2.0
18882430835,2,1,2.0,2,2,2.0,2.0,2.0
18772912512,2,1,2.0,2,2,2.0,2.0,2.0


In [40]:
def ratio(df_func, last_n=2):
    df_temp = df_func.tail(last_n)
    ratio = df_temp['real_sessiontime'].std() / df_temp['real_sessiontime'].mean()
    return ratio

In [41]:
df = df.sort_index()

In [42]:
df_dest_ratio_raw = df.groupby('calledstation')

In [43]:
df_dest_ratio_raw

In [44]:
cont = 0

for num, frame in df_dest_ratio_raw:
    if cont < 100:
        print('**'*50)
        print(f'Número: {num}')
        print(frame.tail(2)['real_sessiontime'])
        stand_dev = frame.tail(2)['real_sessiontime'].std()
        prom = frame.tail(2)['real_sessiontime'].mean()
        print(f'Desv Stand de las últimos dos llamadas: {stand_dev}')
        print(f'Promedio de las últimos dos llamadas: {prom}')
        print(f'Ratio de las últimos dos llamadas calculado: {stand_dev/prom}')
        print(f'Ratio de las últimos dos llamadas función ratio: {ratio(frame)}')
        print(f'Validación: {ratio(frame)==stand_dev/prom}')


        cont += 1
    

****************************************************************************************************
Número: 18002001956
starttime
2024-08-12 20:04:36    583
2024-08-13 17:30:37    595
Name: real_sessiontime, dtype: object
Desv Stand de las últimos dos llamadas: 8.48528137423857
Promedio de las últimos dos llamadas: 589.0
Ratio de las últimos dos llamadas calculado: 0.014406250210931358
Ratio de las últimos dos llamadas función ratio: 0.014406250210931358
Validación: True
****************************************************************************************************
Número: 18002002000
starttime
2024-08-14 09:55:14    39
2024-08-14 14:07:32    38
Name: real_sessiontime, dtype: object
Desv Stand de las últimos dos llamadas: 0.7071067811865476
Promedio de las últimos dos llamadas: 38.5
Ratio de las últimos dos llamadas calculado: 0.018366409900949288
Ratio de las últimos dos llamadas función ratio: 0.018366409900949288
Validación: True
***********************************************

In [45]:
df_dest_ratio = df.groupby('calledstation').apply(ratio)

/tmp/ipykernel_437213/1556550013.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_dest_ratio = df.groupby('calledstation').apply(ratio)


In [46]:
# df_dest_ratio = df_dest_ratio.rename(columns={0:'flat_ratio'})
df_dest_ratio.to_frame()

,0
calledstation,
18002001956,0.014406
18002002000,0.018366
18002002599,0.025713
18002002665,0.850039
18002003111,0.816335
...,...
18889995252,0.833753
18889996569,0.000000
18889999101,0.000000


In [47]:
df_dest_ratio.rename('flat_ratio')

calledstation
18002001956    0.014406
18002002000    0.018366
18002002599    0.025713
18002002665    0.850039
18002003111    0.816335
                 ...   
18889995252    0.833753
18889996569    0.000000
18889999101    0.000000
18889999353    0.132583
18889999520    0.097532
Name: flat_ratio, Length: 50315, dtype: float64

In [48]:
df_dest = df_dest.merge(df_dest_ratio.rename('flat_ratio'), left_index=True, right_index=True)
df_dest

,real_sessiontime,num_llamadas,dur_prom,real_sessiontime_max,real_sessiontime_min,real_sessiontime_P25,real_sessiontime_P50,real_sessiontime_P75,flat_ratio
calledstation,,,,,,,,,
18005514636,21257,7,3036.714286,3600,1897,2486.0,3593.0,3597.5,0.438131
18003703848,21144,7,3020.571429,3600,1767,2490.5,3598.0,3599.0,0.482999
18558856518,21106,7,3015.142857,3601,1774,2467.0,3598.0,3599.5,0.425592
18007456973,21057,7,3008.142857,3602,1661,2501.5,3593.0,3599.0,0.425771
18776981885,20930,7,2990.0,3601,1662,2435.5,3598.0,3599.0,0.450860
...,...,...,...,...,...,...,...,...,...
18885302154,2,1,2.0,2,2,2.0,2.0,2.0,NaN
18882430835,2,1,2.0,2,2,2.0,2.0,2.0,NaN
18772912512,2,1,2.0,2,2,2.0,2.0,2.0,NaN


In [49]:
df_dest_ratio.values

array([0.01440625, 0.01836641, 0.02571297, ..., 0.        , 0.13258252,
       0.09753197])

In [50]:
len(df_dest_ratio)

50315

In [51]:
df_dest_ratio.values * len(df_dest_ratio)

array([ 724.85047936,  924.10591417, 1293.74827983, ...,    0.        ,
       6670.88956789, 4907.32106143])

In [ ]:
df_dest.sort_values(by='real_sessiontime', ascending=False)

In [ ]:
umbral = 0.005
df_dest_flat = df_dest[df_dest.flat_ratio <= umbral].sort_values(by='real_sessiontime', ascending=False)
df_dest_flat

In [ ]:
df_dest.real_sessiontime.sum()/60

In [ ]:
(df_dest_flat.real_sessiontime.sum()/60) / (df_dest.real_sessiontime.sum()/60)

In [ ]:
df_dest_flat.dur_prom.max()

In [ ]:
(df_dest_flat[df_dest_flat.dur_prom >= 600].real_sessiontime.sum()/60) / (df_dest.real_sessiontime.sum()/60)

In [ ]:
index_lm = df_dest_flat.index
index_lm

In [ ]:
bd

In [ ]:
cursores

In [ ]:
datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
block_numbers = ''
ind = 0
num_block = 10


while ind < num_block:
    block_numbers += "("
    block_numbers += index_lm[ind]
    block_numbers += ","
    block_numbers += f"'Número agregado por Carlos Gorricho desde script the Python el {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}'"
    if ind < num_block - 1:
        block_numbers += "),\n"
    else:
        block_numbers += ")"

    ind += 1


print(block_numbers)


In [ ]:
# define la consulta (query)
add_query = dedent(f"""INSERT INTO `pkg_campaign_restrict_phone` (number, description) VALUES {block_numbers};""")

print(f'Query a correr:\n{add_query}')


In [ ]:
# ejecuta consulta en todos los marcadores
for key in cursores:
    cursores[key].execute(add_query)
    print(f'Ejectando consulta en {key} - Agregadas {cursores[key].rowcount} filas')
    


In [ ]:
# compromete los cambios en todos los marcadores
for base in bd:
    bd[base].commit()
    print(f'Condirmando cambios en {base}')
    

### Visualizaciones

#### Flatliners

In [ ]:
df

In [ ]:
index_lm = df_dest_flat.index
index_lm

In [ ]:
rows=5
cols=5
indice=0
i=0
index_lm[i+indice*rows*cols]

In [ ]:
df_dest.index[i+indice*rows*cols]

In [619]:
def plot_flat_ts(df, ind=0):    
    rows = 5
    cols = 5
    indice = ind
    fix, ax = plt.subplots(rows,cols,
                        figsize=(20,rows * 4),
                        sharey=True,
                        sharex=True,
                        squeeze=False)

    for i in range(rows * cols):
        ax[i//5, i%5].plot(df[df['calledstation']==index_lm[i+indice*rows*cols]].real_sessiontime)
        ax[i//5, i%5].set_title(f"{i+1+indice*rows*cols}. {df_dest_flat.index[i+indice*rows*cols]} ({df_dest_flat.loc[index_lm[i+indice*rows*cols], 'flat_ratio']*100:.3f}%)\n")
        ax[i//5, i%5].set_xticklabels([])

    plt.show()


In [620]:
def plot_flat(df, ind=0):    
    rows = 5
    cols = 5
    indice = ind
    fix, ax = plt.subplots(rows,cols,
                        figsize=(20,rows * 4),
                        sharey=True,
                        sharex=False,
                        squeeze=False)

    for i in range(rows * cols):
        ax[i//5, i%5].plot(df[df['calledstation']==index_lm[i+indice*rows*cols]].real_sessiontime)
        ax[i//5, i%5].set_title(f"{i+1+indice*rows*cols}. {df_dest_flat.index[i+indice*rows*cols]} ({df_dest_flat.loc[index_lm[i+indice*rows*cols], 'flat_ratio']*100:.3f}%)\n")
        ax[i//5, i%5].set_xticklabels([])

    plt.show()


In [621]:
indice=0

In [ ]:
plot_flat(df, ind=indice)

In [ ]:
%%time
plot_flat_ts(df, ind=indice)

In [ ]:
index_lm[:20]

In [ ]:
df_grafica = df[df.calledstation.isin(index_lm[i+indice*rows*cols:i+indice*rows*cols+rows*cols])]
df_grafica

In [626]:
unique_nums = df_grafica.calledstation.unique()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10),
                        sharex=True)

for num in unique_nums:
    df_line = df_grafica[df_grafica.calledstation == num]
    ax.plot(df_line.index, df_line.real_sessiontime)
    pass

#### Comportamiento de llamadas en el tiempo

In [ ]:
df.info()

In [ ]:
df.groupby('hour').count().calledstation.plot(kind='bar')

In [ ]:
# lunes = 0, domingo = 6
df.groupby('weekday').count().calledstation.plot(kind='bar')

#### Duración de llamadas en el tiempo

In [631]:
import seaborn as sns

In [ ]:
index_lm

In [654]:
dest_num = '18889041203'

In [ ]:
df.info()

In [ ]:
dest_num = '18552519301'

fix, ax1 = plt.subplots(figsize=(5,5))

plt.xticks(rotation='vertical')

sns.scatterplot(data = df[df.calledstation == dest_num],
             x = df[df.calledstation == dest_num].index,
             y = 'real_sessiontime',
             hue = 'bd',
             s=50,                         
             ax=ax1,
             )

ax2 = ax1.twinx()

sns.lineplot(data = df[df.calledstation == dest_num],
             x = df[df.calledstation == dest_num].index,
             y = 'real_sessiontime',
             ax=ax2,
             )

plt.show()

In [ ]:
df[df.calledstation == dest_num]

In [ ]:
low = 0
hi = 2
df[df.calledstation == dest_num].real_sessiontime.iloc[low:hi].std() / df[df.calledstation == dest_num].real_sessiontime.iloc[low:hi].mean()

#### Dispersión de duración de llamadas según frecuencia

In [ ]:
sns.lineplot(data = df[df.calledstation == dest_num],
             x = df_dest.num_llamadas,
             y = df_dest.dur_prom,
             errorbar=('ci', 100),
             )

#### Frecuencia de número de llamadas

In [ ]:
df_dest.num_llamadas.value_counts()

In [ ]:
df_dest.num_llamadas.value_counts().plot(kind='bar')

#### Histogramas

In [ ]:
# por duraciones promedio
df_dest.dur_prom.hist(bins=100)

In [ ]:
# por duraciones reales
df.real_sessiontime.hist(bins=100)

### Simulación

In [ ]:
# Define the start time and the frequency (5 minutes)
start_time = '2024-08-01'  # You can choose any start date
end_time = '2024-08-30'    # One week later
freq = '5T'  # 'T' stands for minutes, so '5T' is every 5 minutes

# Generate the time series
time_series = pd.date_range(start=start_time, end=end_time, freq=freq)

# Convert to DataFrame (optional, if you need a DataFrame)
df_sim = pd.DataFrame(time_series, columns=['Timestamp'])

# Display the first few entries
df_sim.head()


In [ ]:
df_sim.tail()